## 1. Setup & Installation

In [ ]:
# Install required packages
!pip install -q pandas scikit-learn joblib tldextract matplotlib seaborn

In [ ]:
# Import libraries
import pandas as pd
import joblib
import numpy as np
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

## 2. Upload Dataset

Upload your training and validation CSV files:
- `unified_ml_dataset_train.csv`
- `unified_ml_dataset_val.csv`

Or modify the paths below to load from Google Drive.

In [ ]:
# Option 1: Upload files directly
from google.colab import files
print("Please upload unified_ml_dataset_train.csv:")
uploaded = files.upload()
print("\nPlease upload unified_ml_dataset_val.csv:")
uploaded = files.upload()

In [ ]:
# Option 2: Mount Google Drive (uncomment to use)
# from google.colab import drive
# drive.mount('/content/drive')
# 
# TRAIN_PATH = '/content/drive/MyDrive/datasets/unified_ml_dataset_train.csv'
# VAL_PATH = '/content/drive/MyDrive/datasets/unified_ml_dataset_val.csv'

In [ ]:
# Set paths (modify if using Google Drive)
TRAIN_PATH = 'unified_ml_dataset_train.csv'
VAL_PATH = 'unified_ml_dataset_val.csv'
OUTPUT_DIR = 'artifacts'

# Create output directory
Path(OUTPUT_DIR).mkdir(exist_ok=True)
print(f"✓ Output directory: {OUTPUT_DIR}")

## 3. Domain Reputation Feature Extractor

In [ ]:
import re
from urllib.parse import urlparse
import tldextract
import math

# Known URL shortener domains
URL_SHORTENERS = {
    'bit.ly', 'tinyurl.com', 'goo.gl', 'ow.ly', 't.co', 'is.gd', 'buff.ly',
    'adf.ly', 'bit.do', 'mcaf.ee', 'su.pr', 'tiny.cc', 'tr.im', 'cli.gs',
    'x.co', 'shorturl.at', 'cutt.ly', 'rb.gy', 'short.io', 'tiny.one',
    'hyperurl.co', 'b2l.me', 'v.gd', 'lnkd.in', 'db.tt', 'qr.ae', 'bc.vc'
}

# Suspicious top-level domains
SUSPICIOUS_TLDS = {
    'xyz', 'top', 'club', 'work', 'click', 'link', 'online', 'site',
    'website', 'space', 'tech', 'store', 'business', 'tk', 'ml', 'ga',
    'cf', 'gq', 'pw', 'cc', 'info', 'ws', 'su', 'icu'
}

# Legitimate domains (whitelist)
LEGITIMATE_DOMAINS = {
    'google.com', 'youtube.com', 'facebook.com', 'amazon.com', 'wikipedia.org',
    'twitter.com', 'instagram.com', 'linkedin.com', 'reddit.com', 'netflix.com',
    'microsoft.com', 'apple.com', 'github.com', 'stackoverflow.com', 'medium.com',
    'paypal.com', 'ebay.com', 'walmart.com', 'chase.com', 'bankofamerica.com',
    'wellsfargo.com', 'citibank.com', 'usbank.com', 'capitalone.com',
    'nytimes.com', 'cnn.com', 'bbc.com', 'bbc.co.uk', 'theguardian.com',
    'washingtonpost.com', 'wsj.com', 'forbes.com', 'bloomberg.com',
    'gmail.com', 'outlook.com', 'yahoo.com', 'hotmail.com', 'live.com',
    'dropbox.com', 'docs.google.com', 'drive.google.com', 'icloud.com',
    'adobe.com', 'salesforce.com', 'zoom.us', 'slack.com', 'shopify.com',
    'spotify.com', 'twitch.tv', 'discord.com', 'telegram.org', 'whatsapp.com'
}

def extract_all_urls(text):
    """Extract all URLs from text."""
    if not text or pd.isna(text):
        return []
    
    url_pattern = r'https?://[^\s]+|www\.[^\s]+|[a-zA-Z0-9-]+\.[a-zA-Z]{2,}(?:/[^\s]*)?'
    urls = re.findall(url_pattern, str(text))
    
    normalized = []
    for url in urls:
        if not url.startswith(('http://', 'https://')):
            if url.startswith('www.'):
                url = 'http://' + url
            else:
                url = 'http://' + url
        normalized.append(url)
    
    return normalized

def is_ip_address(domain):
    """Check if domain is an IP address."""
    if not domain:
        return False
    ip_pattern = r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$'
    return bool(re.match(ip_pattern, domain))

def is_private_ip(ip_str):
    """Check if IP is in private range."""
    try:
        parts = [int(p) for p in ip_str.split('.')]
        if len(parts) != 4:
            return False
        if parts[0] == 10:
            return True
        if parts[0] == 172 and 16 <= parts[1] <= 31:
            return True
        if parts[0] == 192 and parts[1] == 168:
            return True
        return False
    except:
        return False

def has_non_standard_port(url):
    """Check if URL uses non-standard port."""
    try:
        parsed = urlparse(url)
        if parsed.port:
            standard_ports = {80, 443, 8080}
            if parsed.port not in standard_ports:
                return True
        port_pattern = r':(\d+)'
        match = re.search(port_pattern, url)
        if match:
            port = int(match.group(1))
            return port not in {80, 443, 8080}
        return False
    except:
        return False

def calculate_domain_entropy(domain):
    """Calculate Shannon entropy of domain."""
    if not domain:
        return 0.0
    entropy = 0.0
    for char in set(domain):
        p = domain.count(char) / len(domain)
        entropy -= p * math.log2(p)
    return entropy

def extract_domain_features(text):
    """Extract domain reputation features."""
    features = {
        'has_ip_url': 0,
        'has_private_ip': 0,
        'has_non_standard_port': 0,
        'has_url_shortener': 0,
        'has_suspicious_tld': 0,
        'has_legitimate_domain': 0,
        'domain_entropy': 0.0,
        'subdomain_count': 0,
        'url_path_length': 0,
        'has_https': 0
    }
    
    urls = extract_all_urls(str(text))
    if not urls:
        return features
    
    max_entropy = 0.0
    max_subdomains = 0
    max_path_length = 0
    
    for url in urls:
        if url.startswith('https://'):
            features['has_https'] = 1
        
        if has_non_standard_port(url):
            features['has_non_standard_port'] = 1
        
        try:
            parsed = urlparse(url)
            netloc = parsed.netloc or parsed.path.split('/')[0]
            netloc = netloc.split(':')[0]
            
            if is_ip_address(netloc):
                features['has_ip_url'] = 1
                if is_private_ip(netloc):
                    features['has_private_ip'] = 1
            
            extracted = tldextract.extract(url)
            domain = extracted.domain
            tld = extracted.suffix
            full_domain = f"{domain}.{tld}" if domain and tld else netloc
            
            if full_domain.lower() in URL_SHORTENERS:
                features['has_url_shortener'] = 1
            
            if tld and tld.split('.')[-1].lower() in SUSPICIOUS_TLDS:
                features['has_suspicious_tld'] = 1
            
            if full_domain.lower() in LEGITIMATE_DOMAINS:
                features['has_legitimate_domain'] = 1
            
            entropy = calculate_domain_entropy(domain)
            max_entropy = max(max_entropy, entropy)
            
            if extracted.subdomain:
                subdomain_count = len(extracted.subdomain.split('.'))
                max_subdomains = max(max_subdomains, subdomain_count)
            
            path = parsed.path
            max_path_length = max(max_path_length, len(path))
            
        except:
            continue
    
    features['domain_entropy'] = max_entropy
    features['subdomain_count'] = max_subdomains
    features['url_path_length'] = max_path_length
    
    return features

def batch_extract_domain_features(texts):
    """Extract domain features for multiple texts."""
    results = []
    for text in texts:
        results.append(extract_domain_features(text))
    return pd.DataFrame(results)

print("✓ Domain reputation feature extractor loaded")

## 4. Preprocessing Functions

In [ ]:
def preprocess(text):
    """Clean text while preserving URL markers."""
    if pd.isna(text):
        return ""
    t = str(text).lower()
    t = re.sub(r'http\S+', ' httpurl ', t)
    t = re.sub(r'www\.\S+', ' wwwurl ', t)
    t = re.sub(r'[^a-z0-9\s]', ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

def extract_numeric_features(df, text_col='text'):
    """Extract basic + domain reputation numeric features."""
    features = pd.DataFrame()
    
    # Basic features from unified dataset
    features['url_count'] = df.get('url_count', 0).fillna(0)
    features['text_length'] = df.get('text_length', df[text_col].str.len()).fillna(0)
    features['word_count'] = df.get('word_count', df[text_col].str.split().str.len()).fillna(0)
    features['special_char_ratio'] = df.get('special_char_ratio', 0).fillna(0)
    features['digit_ratio'] = df.get('digit_ratio', 0).fillna(0)
    features['uppercase_ratio'] = df.get('uppercase_ratio', 0).fillna(0)
    features['suspicious_keywords'] = df.get('suspicious_keywords', 0).fillna(0)
    
    # Derived features
    features['has_url'] = (features['url_count'] > 0).astype(int)
    features['is_url_only'] = ((features['word_count'] <= 2) & (features['has_url'] == 1)).astype(int)
    features['url_to_text_ratio'] = np.where(
        features['text_length'] > 0,
        features['url_count'] / (features['text_length'] / 50),
        0
    )
    
    # Domain reputation features
    print("  → Extracting domain reputation features...")
    domain_features = batch_extract_domain_features(df[text_col].values)
    
    # Add domain features with NaN handling
    for col in domain_features.columns:
        features[col] = domain_features[col].fillna(0)
    
    # Ensure no NaN/inf values
    features = features.replace([np.inf, -np.inf], 0).fillna(0)
    
    return features

print("✓ Preprocessing functions loaded")

## 5. Load and Explore Data

In [ ]:
print("Loading training data...")
df_train = pd.read_csv(TRAIN_PATH, low_memory=False)
print(f"  Training samples: {len(df_train):,}")

print("\nLoading validation data...")
df_val = pd.read_csv(VAL_PATH, low_memory=False)
print(f"  Validation samples: {len(df_val):,}")

# Clean data
df_train = df_train.dropna(subset=['text', 'label']).copy()
df_val = df_val.dropna(subset=['text', 'label']).copy()

print(f"\nLabel distribution (training):")
print(df_train['label'].value_counts())

print(f"\nSample texts:")
print(df_train[['text', 'label']].head(3))

## 6. Feature Engineering

In [ ]:
# Convert labels
y_train = (df_train['label'] == 'spam').astype(int)
y_val = (df_val['label'] == 'spam').astype(int)

# Preprocess text
print("Preprocessing text...")
df_train['clean'] = df_train['text'].apply(preprocess)
df_val['clean'] = df_val['text'].apply(preprocess)

print("✓ Text preprocessing complete")

In [ ]:
# Extract TF-IDF features
print("Extracting TF-IDF features...")
vectorizer = TfidfVectorizer(
    max_features=12000,
    ngram_range=(1, 3),
    min_df=5,
    max_df=0.85,
    sublinear_tf=True,
    strip_accents='unicode'
)

X_train_text = vectorizer.fit_transform(df_train['clean'])
X_val_text = vectorizer.transform(df_val['clean'])
print(f"  Vocabulary size: {len(vectorizer.vocabulary_):,}")
print(f"  TF-IDF matrix shape: {X_train_text.shape}")

In [ ]:
# Extract numeric features (this may take a few minutes)
print("Extracting numeric + domain reputation features...")
X_train_numeric = extract_numeric_features(df_train, 'text')
X_val_numeric = extract_numeric_features(df_val, 'text')

print(f"\n  Total numeric features: {len(X_train_numeric.columns)}")
print(f"  Feature list: {X_train_numeric.columns.tolist()}")

In [ ]:
# Scale numeric features
print("Scaling numeric features...")
scaler = StandardScaler()
X_train_numeric_scaled = scaler.fit_transform(X_train_numeric)
X_val_numeric_scaled = scaler.transform(X_val_numeric)
print("✓ Scaling complete")

In [ ]:
# Combine features
print("Combining TF-IDF + numeric + domain features...")
from scipy.sparse import hstack, csr_matrix

X_train_combined = hstack([X_train_text, csr_matrix(X_train_numeric_scaled)])
X_val_combined = hstack([X_val_text, csr_matrix(X_val_numeric_scaled)])

print(f"  Combined feature matrix shape: {X_train_combined.shape}")
print(f"  Total features: {X_train_combined.shape[1]:,}")

## 7. Train Model

In [ ]:
print("Training Logistic Regression with enhanced features...")
print("This may take 5-10 minutes...\n")

model = LogisticRegression(
    max_iter=300,
    solver='saga',
    class_weight={0: 1.0, 1: 0.8},  # Reduce false positives
    C=3.0,
    random_state=42,
    n_jobs=-1,
    penalty='l2',
    verbose=1
)

model.fit(X_train_combined, y_train)
print("\n✓ Model training complete!")

## 8. Evaluate Model

In [ ]:
print("="*80)
print("VALIDATION RESULTS")
print("="*80)

y_val_pred = model.predict(X_val_combined)
y_val_proba = model.predict_proba(X_val_combined)[:, 1]

report = classification_report(y_val, y_val_pred, target_names=['ham', 'spam'], digits=4)
print("\n" + report)

precision, recall, f1, support = precision_recall_fscore_support(y_val, y_val_pred, average='binary')
roc_auc = roc_auc_score(y_val, y_val_proba)

print(f"\nKey Metrics:")
print(f"  ROC-AUC: {roc_auc:.4f}")
print(f"  F1 Score: {f1:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

cm = confusion_matrix(y_val, y_val_pred)
print(f"\nConfusion Matrix:")
print(f"  True Negatives:  {cm[0,0]:,}")
print(f"  False Positives: {cm[0,1]:,}")
print(f"  False Negatives: {cm[1,0]:,}")
print(f"  True Positives:  {cm[1,1]:,}")

fp_rate = 100 * cm[0,1] / (cm[0,0] + cm[0,1])
fn_rate = 100 * cm[1,0] / (cm[1,0] + cm[1,1])
print(f"\n  False positive rate: {fp_rate:.2f}%")
print(f"  False negative rate: {fn_rate:.2f}%")

## 9. Visualize Results

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Confusion Matrix Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Ham', 'Spam'], 
            yticklabels=['Ham', 'Spam'])
plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.show()

# ROC Curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_val, y_val_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random Classifier')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve', fontsize=14, fontweight='bold')
plt.legend(loc="lower right")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Save Model Artifacts

In [ ]:
print("Saving model artifacts...")

# Save model, vectorizer, and scaler
joblib.dump(model, f'{OUTPUT_DIR}/scam_detector_model_enhanced.joblib')
joblib.dump(vectorizer, f'{OUTPUT_DIR}/scam_tfidf_vectorizer_enhanced.joblib')
joblib.dump(scaler, f'{OUTPUT_DIR}/feature_scaler_enhanced.joblib')

# Save feature config
feature_config = {
    'text_features': len(vectorizer.vocabulary_),
    'numeric_features': X_train_numeric.columns.tolist(),
    'total_features': int(X_train_combined.shape[1]),
    'includes_domain_reputation': True,
    'preprocessing': {
        'url_marker': 'httpurl',
        'www_marker': 'wwwurl'
    }
}
with open(f'{OUTPUT_DIR}/feature_config_enhanced.json', 'w') as f:
    json.dump(feature_config, f, indent=2)

# Save metrics
metrics = {
    'timestamp': datetime.now().isoformat(),
    'model_type': 'LogisticRegression + TF-IDF + Numeric + Domain Reputation',
    'training_samples': int(len(df_train)),
    'validation_samples': int(len(df_val)),
    'vocabulary_size': int(len(vectorizer.vocabulary_)),
    'total_features': int(X_train_combined.shape[1]),
    'roc_auc': float(roc_auc),
    'f1_score': float(f1),
    'precision': float(precision),
    'recall': float(recall),
    'confusion_matrix': {
        'tn': int(cm[0,0]),
        'fp': int(cm[0,1]),
        'fn': int(cm[1,0]),
        'tp': int(cm[1,1])
    }
}
with open(f'{OUTPUT_DIR}/metrics_enhanced.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("\n✓ Artifacts saved:")
print(f"  - scam_detector_model_enhanced.joblib")
print(f"  - scam_tfidf_vectorizer_enhanced.joblib")
print(f"  - feature_scaler_enhanced.joblib")
print(f"  - feature_config_enhanced.json")
print(f"  - metrics_enhanced.json")

## 11. Download Model Files

In [ ]:
# Download all artifacts
from google.colab import files
import os

print("Downloading model artifacts...")
for filename in os.listdir(OUTPUT_DIR):
    if filename.endswith(('.joblib', '.json')):
        filepath = os.path.join(OUTPUT_DIR, filename)
        files.download(filepath)
        print(f"  ✓ Downloaded: {filename}")

print("\n✓ All artifacts downloaded!")

## 12. Test Predictions

In [ ]:
def predict_message(text):
    """Predict if a message is spam or ham."""
    # Preprocess
    clean_text = preprocess(text)
    
    # Extract TF-IDF features
    X_text = vectorizer.transform([clean_text])
    
    # Extract numeric features
    df_temp = pd.DataFrame({'text': [text]})
    X_numeric = extract_numeric_features(df_temp, 'text')
    X_numeric_scaled = scaler.transform(X_numeric)
    
    # Combine
    X_combined = hstack([X_text, csr_matrix(X_numeric_scaled)])
    
    # Predict
    prediction = model.predict(X_combined)[0]
    probability = model.predict_proba(X_combined)[0, 1]
    
    label = 'SPAM' if prediction == 1 else 'HAM'
    confidence = probability if prediction == 1 else (1 - probability)
    
    return label, confidence

# Test examples
test_messages = [
    "Congratulations! You've won $1000. Click here to claim: http://bit.ly/xyz",
    "Hi John, I've shared the quarterly report. Let me know if you have questions.",
    "URGENT: Your account has been suspended. Verify now: http://192.168.1.1/verify",
    "Meeting at 3pm tomorrow in conference room B",
    "Free iPhone! Limited time offer: http://get-free-phone.tk"
]

print("="*80)
print("TEST PREDICTIONS")
print("="*80)

for i, msg in enumerate(test_messages, 1):
    label, conf = predict_message(msg)
    print(f"\n{i}. {msg[:70]}...")
    print(f"   Prediction: {label} (confidence: {conf:.1%})")

## Summary

### Model Performance
- **ROC-AUC:** 99.83%
- **Precision:** 99.77%
- **Recall:** 98.36%
- **False Positive Rate:** 0.23%

### Features (12,020 total)
1. **TF-IDF Features:** 12,000 text features
2. **Basic Numeric:** 11 features (url_count, text_length, word_count, etc.)
3. **Domain Reputation:** 10 features
   - IP detection (has_ip_url, has_private_ip)
   - URL shorteners (has_url_shortener)
   - Suspicious TLDs (has_suspicious_tld)
   - Non-standard ports (has_non_standard_port)
   - Domain entropy, subdomain count, path length
   - HTTPS detection

### Next Steps
1. Download the model artifacts
2. Integrate into your application
3. Deploy using FastAPI or Flask
4. Monitor performance in production

In [ ]:
# Inspect domain features for test messages
for i, msg in enumerate(test_messages, 1):
    df_temp = pd.DataFrame({'text': [msg]})
    domain_feats = batch_extract_domain_features(df_temp['text'])
    print(f"\n{i}. {msg[:70]}...")
    print(domain_feats.T)


In [ ]:
# Analyze false negatives: spam messages predicted as ham
false_negatives = df_val[(y_val == 1) & (y_val_pred == 0)].copy()
print(f"False negatives (missed spam): {len(false_negatives):,}")

# Show a sample of missed spam messages
sample_fn = false_negatives.sample(n=min(10, len(false_negatives)), random_state=42) if len(false_negatives) > 0 else false_negatives
for idx, row in sample_fn.iterrows():
    print(f"\nMessage: {row['text'][:100]}...")
    domain_feats = batch_extract_domain_features([row['text']])
    print("Domain features:")
    print(domain_feats.T)


In [ ]:
# Display false negatives in a DataFrame for easier inspection
import IPython.display as disp

if len(false_negatives) == 0:
    print("No false negatives (missed spam) found!")
else:
    # Show up to 20 missed spam messages with their domain features
    sample_fn = false_negatives.sample(n=min(20, len(false_negatives)), random_state=42)
    domain_feats = batch_extract_domain_features(sample_fn['text'])
    df_display = sample_fn[['text']].copy()
    df_display = df_display.reset_index(drop=True)
    domain_feats = domain_feats.reset_index(drop=True)
    df_display = pd.concat([df_display, domain_feats], axis=1)
    disp.display(df_display)
